### It is one part of an ETL process that read three different kinds of .txt from eight different companies and finally send all information to SQL Server where the files are consolidated, to perform the following:

* Balance Sheet
* Profit and Loss
* Cash Flow
* Ad-hoc reports
* KPI's

hernantorres23@mail.ru  
https://www.linkedin.com/in/hernantorres23/

In [1]:
import platform 
print(f'Python Version = {platform.python_version()}') 
# print(platform.python_version_tuple())


Python Version = 3.8.2


In [2]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from pathlib import Path
import urllib
import os
import getpass
# import sharepy
import socket
import requests
import sys
from datetime import datetime

from internetconn import have_internet

### Configurar los parametros en cada caso
### Set the parameters for each case

In [3]:
user_home = os.path.expanduser('~')                    # tambien sirve os.environ['USERPROFILE']
user_name = os.path.split(user_home)[-1].split('.')[0] # tambien sirve os.environ['USERNAME'] ó getpass.getuser() ó os.environ.get('USERNAME')
file_to_read = 'comp1.txt'                             # esto es variable, segun la seleccion del usuario
report_folder = 'Consolidacion BI'                     # esto es una regla (peticion de la empresa)
full_report_path = os.path.join(user_home, 'Desktop', report_folder)
sharepoint_site = 'https://empresa.sharepoint.com'
company_prefix = 'H'                                   # El prefijo diferenciador en el DW, variable segun la seleccion del usuario
company_abrv_name = 'COMPANY1'                         # El nombre abreviado o siglas de la empresa 
company_code = 'COMP1'                                 # El codigo de la compañia a nivel de la consolidación
source_system = 'SAINT'                                # El sistema administrativo de origen
sharepoint_company_folder = '/'.join(['Consolidado', source_system, company_abrv_name])
full_sharepoint_path = '/'.join([sharepoint_site, sharepoint_company_folder, file_to_read])

if have_internet():
    print(f'Los datos seran leidos desde: {full_sharepoint_path}', \
          f'hacia: {full_report_path}', sep = '\n', end = '\n\n')
else:
    print('No tiene conexion', 'Contacte al administrador del sistema', sep='\n\n')
#     sys.exit()  


# print(user_home, user_name, file_to_read, full_report_path, \
#       sharepoint_site, company_prefix, company_abrv_name, \
#       company_code, source_system, sharepoint_company_folder, \
#       full_sharepoint_path, sep='\n')    

Conectado
Los datos seran leidos desde: https://empresa.sharepoint.com/Consolidado/SAINT/COMPANY1/comp4.txt
hacia: C:\Users\hernan\Desktop\Consolidacion BI



### Utilizar la libreria sharepy para conectar con el servidor Sharepoint de la empresa
Esta comentada ya que requiere una conexion real con el servidor para funcionar  

### Using the sharepy library to connect with Sharepoint
Code is commented because raises an error if can not establish the connection

In [4]:
# sharepoint_user = 'usuario.prueba@dominio-ve.com' # credenciales del usuario para conectar: usuario
# active_directory_pass = 'clave065'                # credenciales del usuario para conectar: password

# # intento traer el archivo desde sharepoint en tres etapas
# try:
#     # (1) Authenticate
#     s = sharepy.connect(sharepoint_site,
#                         username=sharepoint_user,
#                         password=clave)

#     # (2) Get file information
#     url_to_folder = "/sites/COETUS360/_api/web/GetFolderByServerRelativeUrl('"
#     r = s.get(sharepoint_site + url_to_folder + sharepoint_company_folder + "')/Files")
#     print("Carpeta encontrada...")

#     # (3) Download file
#     url_to_file = "/sites/COETUS360/_api/web/GetFileByServerRelativeUrl('/sites/COETUS360/"
#     r = s.getfile(sharepoint_site + url_to_file + \
#                   '/'.join([sharepoint_company_folder, file_to_read]) + \
#                   "')/$value", filename='/'.join([full_report_path, file_to_read]))
#     print("Archivo descargado...")

# except TimeoutError:
#     print('Demasiado tiempo para establecer la conexion con el servicio Sharepoint')
# except requests.exceptions.ConnectionError:
#     print('No se ha podido establecer la conexion con el servicio Sharepoint')

### Validar si el directorio de trabajo existe y sino debe crearse
### Validate the working directory exists and create it if not

In [5]:
'''
Crear el directorio de trabajo en caso de no existir o que haya sido eliminado por el usuario
Create the working directory if not exists or when, by mistake, was removed by the user.
'''
if os.path.isdir(full_report_path):
    print('El directorio de trabajo ya existia!!!')
else:
    path = Path(full_report_path)
    path.mkdir(exist_ok=True)
    print(f'Se creo el directorio de trabajo en {full_report_path}')

El directorio de trabajo ya existia!!!


### Crear el engine para conectarse a la base de datos y validar configuracion del administrador del proyecto
### Set an engine to connect the database and to validate the settings created by the project chief.

In [6]:
try:
    # Utilizar credenciales de Windows
    # Using Windows credentials
    # quoted = urllib.parse.quote_plus("DRIVER={SQL Server};SERVER=Rexandmikel\MIKEL;DATABASE=github_sqlserver;Trusted_Connection=yes")
    
    # Using SQL Server credentials
    quoted = urllib.parse.quote_plus("DRIVER={SQL Server}; SERVER=Rexandmikel\\MIKEL; \
                                      DATABASE=github_sqlserver; Trusted_Connection=no; \
                                      USER=hernan; PWD=123456")
    engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

    # determinación del periodo vigente
    # Cheking the current period
    qry_periodo = "Select TOP 1 id from periods_planner Where type='P'"
    periods = pd.read_sql(qry_periodo, con=engine, index_col=None,
                           coerce_float=True, parse_dates=None,
                           columns=None, chunksize=None)
    if periods.empty:
        print('No se ha establecido un periodo para recibir los datos') 
    else:
        print('\n---------INFORMATION---------', '|----Conexion exitosa con el servidor de datos----|', sep='\n\n', end='\n\n')
        # print(type(periods), periods.shape)
        # print(periods.iloc[0,0][1:])
        # periods.set_index(['ID'], inplace=True)
        # periods
        # active_period = periods.index[0][1:5] + '-' + periods.index[0][-2:]
        periods.iloc[0,0]
        active_period = periods.iloc[0,0][1:5] + '-' + periods.iloc[0,0][-2:]
        print(f'El periodo activo es: {active_period}', end = '\n\n')
        # periods.head()

    # determinación de los usuarios que se van a cargar en el combobox
    # Checking how many users will be loaded in the GUI 
    qry_users = "Select user_name,f_name,l_name,last_login from users_master \
    Where security_level != 0"
    #print(qry_users)
    users = pd.read_sql(qry_users, con=engine, index_col=None,
                           coerce_float=True, parse_dates=None,
                           columns=None, chunksize=None)
    if users.empty:
        print('No se han asignado usuarios al proyecto')
    else:
        users.set_index(['user_name'], inplace=True)
        print(f'Los usuarios activos en el proyecto son:', f'{users}', sep = '\n', end = '\n\n')
        # users.head()

    # Sumar los usuarios conectados y si pasan de 5 no permitir el ingreso
    # Sum the concurrent users and to check if the result is greater than five
    qry_current = "Select SUM(logged_in) qty from users_master"
    current_users = pd.read_sql(qry_current, con=engine, index_col=None,
                               coerce_float=True, parse_dates=None,
                               columns=None, chunksize=None)
    current_users.set_index(['qty'], inplace=True) # otra forma de sacar el valor
    print(f'Existen {current_users.index[0]} usuarios conectados', sep = '\n', end = '\n\n')

    # determinacion de las empresas que se van a cargar en el combobox
    # Set which companies will be loaded to the combo box in the GUI
    qry_companies = "SELECT A.id, A.label, left(B.parent,5) grupo \
    FROM groups_m A INNER JOIN groups B ON A.ID=B.child WHERE level_number=-1 \
    AND left(B.parent,5) NOT IN ('conso','matri')"
    companies = pd.read_sql(qry_companies, con=engine, index_col=None,
                            coerce_float=True, parse_dates=None,
                            columns=None, chunksize=None)
    companies.set_index(['label'], inplace=True)
    print(f'Las compañias registradas en el proyecto son:', f'{companies}', sep = '\n', end = '\n\n')
except ModuleNotFoundError: 
    print('Please check if the library "pyodbc" is installed in this environment')
except:
    print('Existe un problema para conectar con la base de datos')


---------INFORMATION---------

|----Conexion exitosa con el servidor de datos----|

El periodo activo es: 2018-01

Los usuarios activos en el proyecto son:
              f_name  l_name           last_login
user_name                                        
GUEST        Invited  Torres  2021-03-09 03:54:59
GUEST1     Invited_1  Torres  2021-03-09 03:54:59

Existen 0 usuarios conectados

Las compañias registradas en el proyecto son:
                                        id  grupo
label                                            
Business Company Alpha Inc.          COMP1   GRP1
Business Company Alpha Inc. USD$  COMP1USD  GRP1U
Business Company Beta Inc.           COMP2   GRP2
Business Company Beta Inc. USD$   COMP2USD  GRP2U
Main Company Business Inc             MAIN   GRP1
Main Company Business Inc USD$     MAINUSD  GRP1U



### Darle un nombre a la transaccion en Microsoft Dynamics

### Set the transaction name to Microsoft Dynamics

In [7]:
batch_name = company_abrv_name + '_' + user_name.upper()
print('El nombre de la transaccion en MS Forecaster es:', batch_name)
batch_label = 'added from Mikel ERP bridge'
batch_name

El nombre de la transaccion en MS Forecaster es: COMPANY1_HERNAN


'COMPANY1_HERNAN'

### Cargar el listado de las cuentas contra la cuales se van a comparar los nuevos registros
### Load the account list to compare it with the new entries

In [8]:
'''
En la interfaz definitiva solo se cargan las que tienen que ver con la compañia en cuestion mas las comunes.
Aunque realmente aca no vemos el efecto, siempre se debe cargar lo necesario
'''
# accounts_query = 'account_master' # si quiero toda la tabla
accounts_query = "Select id, label From account_master Where id like '" + company_prefix + "%'"
accounts = pd.read_sql(accounts_query, con=engine, index_col=None,
                      coerce_float=True, parse_dates=None,
                      columns=None, chunksize=None)
# para habilitar compatibilidd con antiguas versiones esto
# compatibility with old versions
#accounts = pd.read_sql_table(table_name='account_master', con = engine, schema='dbo', index_col=None, coerce_float=True,parse_dates=None, columns=None, chunksize=None)
accounts.set_index(['id'], inplace=True)

accounts.head()

,label
id,
H110101,XXXX XXXXXXXXX (X.X)
H110102,XXXX XXXXXXXXXXX (X.X)
H110103,XXXX XXXXX - XXXX XXXXXXXXXXXXXX (X.X)
H110104,"XXXXX XXX XXXXXX, X.X. XXXXX XXXXXXXXX"
H110105,XXXXX XXXXXXXX XX XXXXXXX (X.X)


### Leer el archivo 
### Read the file

In [9]:
# Nota: Tuve que poner encoding ya que daba errores con algunos caracteres
# Note: In this file is important set the encoding to latin-1 to avoid errors

txt_file = os.path.join(full_report_path, file_to_read)
# print(txt_file)
flag = False
try:
    data = pd.read_csv(txt_file, skiprows=None, sep=",", thousands=".", decimal=',',
                       encoding='latin-1', usecols=[0, 1, 6], header=None, quotechar='"',
                       names=('cuenta', 'nombre', 'monto'),
                       skipinitialspace=True,
                       dtype={'cuenta': np.character, 'nombre': np.character})
    print(f'Success: el archivo ({txt_file}) de origen fue leido satisfactoriamente', end='\n\n')
    flag = True
except FileNotFoundError:
    print("Error: No se ha encontrado el archivo de origen")

# print(data.columns.values)    
# if flag:
#     print(data.head())
data.head()

Success: el archivo (C:\Users\hernan\Desktop\Consolidacion BI\comp4.txt) de origen fue leido satisfactoriamente



,cuenta,nombre,monto
0,1,XXXXXX,NaN
1,1.1,XXXXXXX XXXXXXXXXX,NaN
2,1.1.01,XXXXXXXX XX XXXX X XXXXXX,NaN
3,1.1.01.01,XXXX XXXXXXXXX (X.X),"190,02"
4,1.1.01.02,XXXX XXXXXXXXXXX (X.X),"0,19"


### Podemos revisar como estan nuestros datos con algun o todos los metodos
### Validations

In [10]:
data.info()
# data.shape
# data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1189 entries, 0 to 1188
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cuenta  796 non-null    object
 1   nombre  927 non-null    object
 2   monto   930 non-null    object
dtypes: object(3)
memory usage: 28.0+ KB


* `Se observan registros sin numero de cuenta, que no tienen ningun sentido en estados financieros y podemos borrarlos sin problema`
* `Evaluando la cantidad de registros se presumen lineas totalmente en blanco`

### Eliminar todos lo vacios. 
Por regla general debemos confiar en que los datos de origen tienen todo el sentido, es decir, no puede venir una cuenta contable sin nombre en un estado financiero, tampoco una cuenta sin numero, un valor vacio.  

Por ello entendemos que todos los NaN o Null que se general provienen del formato del archivo y por ende podemos eliminar todos esos registros sin problemas   

Pero en este caso yo prefiero validar y no aplicare `dropna` directamente

### Remove all empty
As a general rule, we must trust that the source of data has all the sense, that is, an accounting account without a name can not come in a financial statement, neither an account without a number or an empty value.

For this reason, I understand that all the NaN or Null generated come from the file format, and therefore we can eliminate all those records without problems.

But in this case, I prefer to validate, and I will not apply `dropna` directly

In [11]:
# data.isna().sum()
data.cuenta.isnull().sum()

393

### Obtenemos el indice de todos los nulos en el campo "cuenta" para luego pasarlos en la funcion drop
En este caso lo hare por indexacion booleana. Para ello creare una serie con todos los indices que necesito eliminar y luego los pasare como argumento en la funcion `drop`

### Obtaining the index of all null in the "account" field, to then pass it to drop function as a boolean indexing

In [12]:
account_nulls = data[data.cuenta.isnull()].index
# account_nulls.shape
# account_nulls.index
account_nulls

Int64Index([  21,   22,   23,   27,   28,   29,   35,   36,   37,   47,
            ...
            1179, 1180, 1181, 1182, 1183, 1184, 1185, 1186, 1187, 1188],
           dtype='int64', length=393)

In [13]:
data = data.drop(list(account_nulls), axis=0)
data

,cuenta,nombre,monto
0,1,XXXXXX,NaN
1,1.1,XXXXXXX XXXXXXXXXX,NaN
2,1.1.01,XXXXXXXX XX XXXX X XXXXXX,NaN
3,1.1.01.01,XXXX XXXXXXXXX (X.X),"190,02"
4,1.1.01.02,XXXX XXXXXXXXXXX (X.X),"0,19"
...,...,...,...
1172,8,XXXX,NaN
1173,8.1,XXXXX XX XXXX,NaN
1174,8.1.01,XXXXX XX XXXX,NaN
1175,8.1.01.01,XXXX XXX XXX,"0,00"


### Verifico la eliminacion de los registros en el campo cuenta
### Now I check again the elimination in "account" field

In [14]:
# data.isna().sum()
data.cuenta.isnull().sum()

0

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 796 entries, 0 to 1176
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cuenta  796 non-null    object
 1   nombre  796 non-null    object
 2   monto   667 non-null    object
dtypes: object(3)
memory usage: 24.9+ KB


### Cambiar el formato de la columna monto.
Aca hay varias cosas que no premiten aplicar otras tecnicas, la primera es que el archivo es separado por comas y ademas el separador decimal tambien es la coma adicionalmente el archivo guarda las posiciones con lo cual tenemos caracteres vacios dentro de las comillas dobles.   

Esta situacion no me ha dejado otro camino que el aplicado aqui, sin embargo, seguire investigando para ver que estoy haciendo mal.

### Fix up the format in the "amount" column.

In [16]:
data['monto'] = pd.to_numeric(data['monto'].str.replace(r'[.)]', "").
                              str.replace(',', ".").str.replace('(', "-"),
                              errors='coerce')
# data.dropna(how='any', inplace=True)
# data.drop(data[(data.monto) == 0].index, inplace=True)

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 796 entries, 0 to 1176
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   cuenta  796 non-null    object 
 1   nombre  796 non-null    object 
 2   monto   667 non-null    float64
dtypes: float64(1), object(2)
memory usage: 24.9+ KB


In [18]:
data[data.monto == 0]
# data.info()

,cuenta,nombre,monto
15,1.1.01.13,XXXXX XXXXXXXXX (X.X),0.0
18,1.1.01.16,XXXXX XXX XXXXXX XXXX (X.X.),0.0
19,1.1.01.17,XXXXX XXXXX XXXXX (X.X),0.0
25,1.1.02.01,XXXXXXXXXXX XXXXXXXXXX XXXXXX XXXXXXX,0.0
31,1.1.03.01,XXXXXXX XXX XXXXXX XXXXXXXXXXX,0.0
...,...,...,...
1139,7.2.01.05,XXXXX XXXXXX,0.0
1151,7.2.02.08,XXXXX XXX XXXXXX,0.0
1152,7.2.02.09,XXXXX XXXXXX,0.0
1175,8.1.01.01,XXXX XXX XXX,0.0


### Aplicando la misma metodologia utilizada para las cuentas, obtenemos el indice de todos los nulos en el campo "monto" para luego pasarlos en la funcion drop
Sin embargo, aca no nos funciona la indexacion booleana ya que como veremos en el resultado de ejecutar la fila siguiente, a pesar de mostrar una propiedad lenght de 796, observo indices que no deberian estar ahi, a simple vista veo el 3 y el 4 y aplicando todo el metodo de las cuentas borraria todos los registros.

### Validation that I can't use the same technique to remove the nulls in the "amount" field
However, here the Boolean indexing does not work for us since as we will see in the result of executing the next row, despite showing a property quantity of entries 796, I see indices that should not be there, with the naked eye I see 3 and 4 and applying all the method of the accounts deletes all the records.

In [19]:
amounts_nulls = data.monto.isna().index
# amounts_nulls.shape
# amounts_nulls.index
amounts_nulls

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            1152, 1159, 1160, 1161, 1162, 1172, 1173, 1174, 1175, 1176],
           dtype='int64', length=796)

In [20]:
data.head()

,cuenta,nombre,monto
0,1,XXXXXX,NaN
1,1.1,XXXXXXX XXXXXXXXXX,NaN
2,1.1.01,XXXXXXXX XX XXXX X XXXXXX,NaN
3,1.1.01.01,XXXX XXXXXXXXX (X.X),190.02
4,1.1.01.02,XXXX XXXXXXXXXXX (X.X),0.19


### Comienzo borrando aquellas entradas que me haya arreglado el coerce ejecutado durante la transformacion de los montos
Por lo explicado anteriormente entonces cambio la tecnica de borrado

### Begin applying deletion with those entries fixed by the coerce parameter in the "amount" transformation process
I start deletion with the entries fix-up by the "coerce" parameter in the "amount" field transformation process

In [21]:
data.dropna(subset=['monto'], inplace=True, axis=0)

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 667 entries, 3 to 1176
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   cuenta  667 non-null    object 
 1   nombre  667 non-null    object 
 2   monto   667 non-null    float64
dtypes: float64(1), object(2)
memory usage: 20.8+ KB


### Pero aun tenemos valores que no nos sirven como veremos al ejecutar la fila siguiente.
### But still, we have entries with zero values, as we will see after the execute the next cell.

In [23]:
sum(data.monto == 0)

377

In [24]:
# Los borramos o filtramos ya que son cuentas con valor cero
# We have the options to delete or filter
data.drop(data[(data.monto) == 0].index, inplace=True) # borrado (to remove)
# data = data[data.monto != 0] # filtrado (filtered)


In [25]:
data

,cuenta,nombre,monto
3,1.1.01.01,XXXX XXXXXXXXX (X.X),190.02
4,1.1.01.02,XXXX XXXXXXXXXXX (X.X),0.19
5,1.1.01.03,XXXX XXXXX - XXXX XXXXXXXXXXXXXX (X.X),0.45
6,1.1.01.04,"XXXXX XXX XXXXXX, X.X XXXXX XXXXXXXXX",71.12
7,1.1.01.05,XXXXX XXXXXXXX XX XXXXXXX (X.X),29574.72
...,...,...,...
1148,7.2.02.05,XXXXX XX XXXXXXXXX,0.01
1149,7.2.02.06,XXXXXXX XXXXXX,139.02
1150,7.2.02.07,XXX XXXXXXX,64.08
1161,7.3.01.01,XXXXXXXX XXX XXXXXXXXXX XX XXXXXX,-76432.03


### Validamos el borrado de las filas con montos igual a cero
### Again, validating our delete

In [26]:
sum(data.monto == 0)

0

### Hacemos un test para validar se cumple el principio de la partida doble
### We make a test to validate the double-entry principle

In [27]:
double_entry_test = round(data.monto.sum(),2)
# data.round({"monto": 2})
if double_entry_test != 0:
    print('Please check the double entry value')
else:
    print(f'Excellent all is working good, the test for double entry is {double_entry_test}')

Excellent all is working good, the test for double entry is 0.0


### Limpiar el campo cuenta y agregar la columna que servira para realizar el JOIN(merge) con la tabla que tiene las cuentas validadas en el DW
* Asignar el prefijo a las cuentas segun la compañia seleccionada
* Eliminar caracteres especiales en las cuentas


### Cleaning the "account" field and add the column to perform the Join("merge" in pandas) with the table that has the valid accounts in the DW
* Assing the prefix to the account according to the selected company
* Remove all special characters in the "account" field

In [28]:
def clean_accounts(x):
    return ''.join(filter(lambda c: c in '0123456789', x))

# Test de la funcion
# print(clean_accounts('1.01.01.30.301'))

data['id'] = company_prefix + data.cuenta.apply(clean_accounts)
data

,cuenta,nombre,monto,id
3,1.1.01.01,XXXX XXXXXXXXX (X.X),190.02,H110101
4,1.1.01.02,XXXX XXXXXXXXXXX (X.X),0.19,H110102
5,1.1.01.03,XXXX XXXXX - XXXX XXXXXXXXXXXXXX (X.X),0.45,H110103
6,1.1.01.04,"XXXXX XXX XXXXXX, X.X XXXXX XXXXXXXXX",71.12,H110104
7,1.1.01.05,XXXXX XXXXXXXX XX XXXXXXX (X.X),29574.72,H110105
...,...,...,...,...
1148,7.2.02.05,XXXXX XX XXXXXXXXX,0.01,H720205
1149,7.2.02.06,XXXXXXX XXXXXX,139.02,H720206
1150,7.2.02.07,XXX XXXXXXX,64.08,H720207
1161,7.3.01.01,XXXXXXXX XXX XXXXXXXXXX XX XXXXXX,-76432.03,H730101


In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 290 entries, 3 to 1162
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   cuenta  290 non-null    object 
 1   nombre  290 non-null    object 
 2   monto   290 non-null    float64
 3   id      290 non-null    object 
dtypes: float64(1), object(3)
memory usage: 11.3+ KB


### Limpiar texto en los nombres de las cuentas (Eliminar espacios antes y despues de la cadena)
### Remove not necessary spaces on the left and right in account names

In [30]:
data.nombre = data.nombre.str.strip()

### Asignar el dataframe a la variable de convencion (df) y ordenar columnas como se requieren
### Apply the name convention to the DataFrame ("df") and sort the columns as required

In [31]:
df = data[['id', 'nombre', 'monto', 'cuenta']]
# df.set_index(['ID'], inplace=True)
rows_qty = df.shape[0] # rows quantity
print (f'El dataframe final tiene {rows_qty} registros validos.', end='\n\n')
print(df.head(15))

El dataframe final tiene 290 registros validos.

         id                                   nombre     monto     cuenta
3   H110101                     XXXX XXXXXXXXX (X.X)    190.02  1.1.01.01
4   H110102                   XXXX XXXXXXXXXXX (X.X)      0.19  1.1.01.02
5   H110103   XXXX XXXXX - XXXX XXXXXXXXXXXXXX (X.X)      0.45  1.1.01.03
6   H110104    XXXXX XXX XXXXXX, X.X XXXXX XXXXXXXXX     71.12  1.1.01.04
7   H110105          XXXXX XXXXXXXX XX XXXXXXX (X.X)  29574.72  1.1.01.05
8   H110106            XXXXXXX XXXXX XXXXXXXXX (X.X)   2935.62  1.1.01.06
9   H110107      XXXXX XXXXXXXXXX XX XXXXXXXXX (X.X)    658.00  1.1.01.07
10  H110108                XXXXX XXXXXXXXXXXX (X.X.)    588.36  1.1.01.08
11  H110109                 XXXXX XX XXXXXXXXX (X.X)   1229.40  1.1.01.09
12  H110110                 XXXXX XXXXXXXXXXXX (X.X)      0.01  1.1.01.10
13  H110111                     XXXXXXX XXXXXX (X.X)  10728.31  1.1.01.11
14  H110112  XXXXX XXXXXXXX XX XXXXXXX XXXXXXX (X.X)      0.04 

### Asignar un indice nos facilitara las cosas cuando queramos revisar algo y cuando vayamos a enviar la informacion al servidor de datos

### Apply an index to make it easy to check something, and when we want to send the information towards the DW

In [32]:
# df.reset_index(inplace=True)
# data.cuenta.is_unique
try:
    if df.id.is_unique:
        my_index = df.id.name
        print(f'The index selected is {my_index}')
        try:
            df.set_index(my_index, inplace = True)
            df.head()
            print(f'The new index is {my_index}')
        except KeyError:
            print(f'The index is already')
    else:
        print(f'The selected field ("{df.id.name}") that you are trying to apply as index, does not have unique values')
        
except KeyError:
    print(f'The index is already')

The index selected is id
The new index is id


In [33]:
df.head(10)

,nombre,monto,cuenta
id,,,
H110101,XXXX XXXXXXXXX (X.X),190.02,1.1.01.01
H110102,XXXX XXXXXXXXXXX (X.X),0.19,1.1.01.02
H110103,XXXX XXXXX - XXXX XXXXXXXXXXXXXX (X.X),0.45,1.1.01.03
H110104,"XXXXX XXX XXXXXX, X.X XXXXX XXXXXXXXX",71.12,1.1.01.04
H110105,XXXXX XXXXXXXX XX XXXXXXX (X.X),29574.72,1.1.01.05
H110106,XXXXXXX XXXXX XXXXXXXXX (X.X),2935.62,1.1.01.06
H110107,XXXXX XXXXXXXXXX XX XXXXXXXXX (X.X),658.00,1.1.01.07
H110108,XXXXX XXXXXXXXXXXX (X.X.),588.36,1.1.01.08
H110109,XXXXX XX XXXXXXXXX (X.X),1229.40,1.1.01.09


### Tener la opcion de cargar en el portapapeles una copia del dataframe con formato exportable a Excel y/o exportarlo automaticamente a la carpeta establecida en las reglas

### Have the option to load in the clipboard the DataFrame and to send it automatically to the folder reports set up in the business rules  

In [34]:
identifier = list(datetime.timetuple(datetime.now()))
# print(identifier, datetime.now())
export_file_name = ''.join([str(n) for n in identifier]) + '_' + company_abrv_name + '_' + user_name + '.xlsx'

# df.T.to_clipboard(excel=True) # con transponer activado
# df.to_clipboard(excel=True)   # esto podria tomar algun tiempo si el DataFrame es grande
# print(date_time)
export_file_to = os.path.join(full_report_path, export_file_name)
df.to_excel(export_file_to)

###  Hecemos un merge. Este nos servira para saber si todas las cuentas que van a enviarse tiene un destino en el DW

### Perform a merge to validate if all new entries have their target account set up by the chief in the DW

In [35]:
'''
Conociendo que tenemos el mismo nombre en ambos DataFrame podemos utilizar on='nombre_del_campo' 
en lugar de utilizar los dos argumentos (left_on='nombre_del_campo_izquierda' y right_on='nombre_del_campo_derecha')
'''
df_account_validator = pd.merge(df, accounts, how='left', on='id')
# df_account_validator = pd.merge(df, cuentas, how='left', left_on='id', right_on='id')

selection = df_account_validator[['label', 'monto', 'cuenta']] # taking only the necesary columns from the previous merge
quantity = df_account_validator.shape[0]
validator = df_account_validator[df_account_validator['label'].isna()]

# print(f'ADVERTENCIA: Se encontraron {validator.shape[0]} registros sin un destino asociado en el DW.', 'NOTA: La informacion no se registrara en el DW hasta que el administrador le autorice', sep='\n')

In [36]:
df_account_validator[df_account_validator['label'].isna()]

,nombre,monto,cuenta,label
id,,,,


In [ ]:
# Others ways to do the same

# nuevos2 = selection[terada['LABEL'].notna()].agg(np.size) # the same result of somedf.notna().sum when we have a total match
# print(nuevos2, end='\n\n')

# print('The same result obtained in nuevos2:',selection.notna().sum(), sep='\n', end='\n\n')

# nuevos4 = df_account_validator[df_account_validator['LABEL'].isna()]
# print(f'There are {nuevos4.shape[0]} new entries', end='\n\n')

# print(df_account_validator.notna().sum(), end='\n\n')

# print(df_account_validator[['LABEL', 'monto', 'cuenta']].notna().sum())

### Validar si existen o no registros nuevos
### Perform a validation for new "accounts"

In [37]:
validator
# file_to_read

,nombre,monto,cuenta,label
id,,,,


In [38]:
if validator.shape[0] > 0:
    '''
    Preparamos los registros con un filtro sobre todos aquellos no encontrados.
    Tambien tenemos la opcion de preparar un merge con el parametro "how=right"
    '''
    print(f'Se encontraron {nuevos} registros sin un destino asociado en el DW.', \
          f'Debe crear la(s) cuenta(s) en Microsoft Forecaster para poder insertar los datos del nuevo periodo.', 'Presione Ctrl + V en Excel ó en un editor de texto, para evaluar los registros involucrados.', sep='\n')
    new_accounts = validator.copy()
    # print(type(new_accounts), new_accounts.shape)
    # print(new_accounts.index)
    new_accounts.reset_index(inplace=True)
    # print(new_accounts.index)
    new_accounts.fillna('Not Found', inplace=True)
    new_accounts.rename({'id': 'id a asignar', 'monto': 'Acumulado', 'cuenta': 'Cuenta origen'}, axis=1, inplace=True)
    new_accounts.to_clipboard(excel=True)
    # aca es donde se envian los correos indicando que hay un usuario que esta limitado para conseguir su objetivo
else:
    '''
    Como todo esta correcto podemos mostrar el DataFrame que se va ingresar en el DW.
    y le adicionamos la cuenta origen para que vea que el "ID" es la misma 
    cuenta origen sin caracteres especiales con la letra prefijo
    '''    
    print('No existen cuentas nuevas y/o notificaciones para enviar al administrador')
    print('There aren\'t new entries and neither notifications to send the chief.')
    # aca se prepara el mensaje de notificacion indicand que un usuario acaba de pasar la validacion de su proceso
    # y esta a un paso de enviar la informacion al tabla de transacciones en Microsoft Dynamics
    selection.to_clipboard(excel=True)

No existen cuentas nuevas y/o notificaciones para enviar al administrador
There aren't new entries and neither notifications to send the chief.


### Finalmente podemos enviar los registros a la transaccion en el DW.
1. Validar si existe una transaccion no posteada en el Microsoft Dynamics y si existe se eliminara
2. Enviar el DataFrame al sistema de transacciones en el DW

### Finally we can to send the entries to the DW
1. Check if exists a previous batch not posted in the DW and if exists delete it
2. Send a DataFrame to SQL Server batch

In [39]:
'''
Preparamos una consulta para pasarla al motor, teniendo en cuenta borrar cualquier transaccion con el mismo nombre
o del mismo periodo si fuera el caso, previo a enviar la consulta para insertar los nuevos registros
'''
qry_delinfo = f"Delete From transactions Where batch='{batch_name}' And errors Is Null" 
# print(qry_delinfo)
# qry_delbatch = "Delete FROM transactions WHERE batch IN ('" + batch_name + "') \
# AND errors IS NULL"
# print(qry_delbatch)

#deleting all previous data from the same period and company or any not posted (committed) information
print('Elminando registros')
Session = sessionmaker(bind=engine)
session = Session()
session.execute(qry_delinfo)
session.commit()
session.close()
print('Registros eliminados')

Elminando registros
Registros eliminados


In [40]:
'''
Finalmente se agregan los registros
Finally we add the entries in the database
'''

cols=['batch', 'gm_id', 'am_id', 'pp_id', 'amount', 'label']
cols_rename={'id': 'am_id', 'monto': 'amount'}
# print(type(cols))
batch_to_insert = selection.copy()
batch_to_insert.reset_index(inplace=True)
# batch_to_insert.reset_index(drop=True, inplace=True)
batch_to_insert.drop(['cuenta'], axis=1, inplace=True)
batch_to_insert.rename(cols_rename, axis=1, inplace=True)

# Adding the necessary columns
batch_to_insert['batch'] = batch_name
batch_to_insert['gm_id'] = company_code
batch_to_insert['pp_id'] = periods.iloc[0,0] # In this system periods have a prefix related to process (What if, Actual, Budget)
batch_to_insert['label'] = batch_label
batch_to_insert = batch_to_insert.reindex(columns=cols)

batch_to_insert.set_index(['batch'], inplace= True)

# Insertar un Pandas DataFrame
# Insert a Pandas DataFrame
try:
    batch_to_insert.to_sql('transactions', con=engine, if_exists='append', schema='dbo')
    print('Se cargaron satisfactoriamente los', str(quantity), 'registros')
    # Se envia el mensaje de la cola
    print('Eliminando archivos temporales...')
    # os.remove(txt_file)
    print("Archivos temporales eliminados!!!!")
except:
    print('Los registros existen en la base de datos')


Se cargaron satisfactoriamente los 290 registros
Eliminando archivos temporales...
Archivos temporales eliminados!!!!


In [ ]:
# Bono: Varias formar de tomar el usuario de entorno
# Bonus: Various ways to get the environ username

periodo = periods.iloc[0,0]
print(periodo)
usuario = os.environ.get('USERNAME')
print(usuario)
usuario = os.path.expanduser('~')
usuario = os.path.split(usuario)[-1]
print(usuario)
usuario = os.environ.get('USERDOMAIN')
print(usuario)
usuario = getpass.getuser()
print(usuario)
# usuario = win32api.GetUserName()
# print(usuario)
# usuario = win32api.GetUserNameEx(...)
# print(usuario)